In [58]:
from transformers.trainer_utils import set_seed
from datasets import load_dataset
import warnings
from transformers import AutoTokenizer, BatchEncoding, AutoModel, EvalPrediction, TrainingArguments, Trainer
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from transformers.utils import ModelOutput
from scipy.stats import spearmanr
from datasets import Dataset
from torch.utils.data import DataLoader

In [13]:
warnings.simplefilter('ignore')

In [14]:
# 乱数の固定
set_seed(42)

In [15]:
# データのロード
unsup_train_dataset = load_dataset('llm-book/jawiki-sentences', split="train")

print(unsup_train_dataset)

Generating train split:   0%|          | 0/24387500 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 24387500
})


---
＜ポイント＞  
・textの列しかもっていない   
・2千万のデータ 

---

In [17]:
# 最初の50件を表示
for i, text in enumerate(unsup_train_dataset[:50]['text']):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

---
＜ポイント＞  
・空白の行があるので、これを前処理で除く必要がある。

---

In [20]:
# 空行を削除
unsup_train_dataset = unsup_train_dataset.filter(lambda example: example['text'].strip() != "")



Filter:   0%|          | 0/23048277 [00:00<?, ? examples/s]

In [21]:
unsup_train_dataset[41:45]

{'text': ['また、主にマイクロソフト系では整数の十六進表記に &h を用い、&h0F (十進で15)のように表現する。',
  'SGML、XML、HTMLでは、アンパサンドを使ってSGML実体を参照する。',
  '言語(げんご)は、狭義には「声による記号の体系」をいう。',
  '広辞苑や大辞泉には次のように解説されている。']}

In [23]:
# 100万データをランダムサンプリング
unsup_train_dataset = unsup_train_dataset.shuffle().select(range(1000000))

# 参照によるマッピングではなく、シャッフルしてサンプリングされたデータセットをディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [31]:
# dataの確認
print(unsup_train_dataset)

print()
print("="*50)
print()

# シャッフルされているかの確認
for i, text in enumerate(unsup_train_dataset[:10]['text']):
    print(i, text)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


0 その学校は基本モード手法からアヴァンギャルド系のジャズ学校でリディアン・クロマティック・コンセプトの大家ジョージ・ラッセル(英語版)も時折訪れて指導をしていた。
1 延暦5年(786年)内舎人となったが、3年後の延暦8年(789年)に史都蒙の予言通り、32歳で病により自邸で没した。
2 「あ号作戦後の兵装増備状況調査」によるとあ号作戦(1944年6月)時点での各艦の対空機銃は以下の通りとされている。
3 一方、5月に領土の返還を求めるフランスがタイ領を攻撃し、国際社会への復帰を優先せざるを得ないタイは、1941年に併合した領土の引き渡しに応じ、ナコーン・チャンパーサック県(英語版)(チャンパーサック州)、ピブーンソンクラーム県(英語版)(シェムリアップ州)、プレアタボン県(英語版)(バタンバン州)の3県がフランスに返還された。
4 およそ140万台が生産された。
5 また、海兵寮同期であった山本とは当時からあまりウマが合わず、4歳年長であった日高がしばしば山本を軽んじる態度を見せたためともいわれている。
6 これら初期の探検家や入植者の中で最も有名な者がダニエル・ブーンであり、伝統的にケンタッキー州創設者の一人と考えられている。
7 全米科学アカデミー会員。
8 他の主力製品はガラス繊維を用いた繊維強化プラスチック (FRP) などの複合材料である。
9 また、このうち三浦、Nanami、有末、トリンドルは『めざましテレビ』(フジテレビ)の『MOTTOいまドキ!』にも出演していた。


In [32]:
# 検証データとテストデータの作成(JSTS)

valid_dataset = load_dataset('llm-book/JGLUE', name='JSTS', split='train')
test_dataset = load_dataset('llm-book/JGLUE', name='JSTS', split='validation')

In [34]:
# tokenizer and collete function

# Tokenizer
base_model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/vocab.txt
loading file spiece.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/tokenizer_config.json
loading file tokenizer.json from cache at None


In [66]:
# collate

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [61]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

In [43]:
valid_dataset[:2]

{'sentence_pair_id': ['0', '1'],
 'yjcaptions_id': ['10005_480798-10996-92616', '100124-104404-104405'],
 'sentence1': ['川べりでサーフボードを持った人たちがいます。', '二人の男性がジャンボジェット機を見ています。'],
 'sentence2': ['トイレの壁に黒いタオルがかけられています。', '2人の男性が、白い飛行機を眺めています。'],
 'label': [0.0, 3.799999952316284]}

In [98]:
class SimCSEModel(nn.Module):
    def __init__(self, base_model_name: str, mlp_only_train: bool = False, temperature: float = 0.05):
        super().__init__()

        self.encoder = AutoModel.from_pretrained(base_model_name)
        self.hidden_size = self.encoder.config.hidden_size
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        self.actiovation = nn.Tanh()

        self.mlp_only_train = mlp_only_train
        self.temperature = temperature

    def encode_text(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダーを使って文をベクトル変換"""

        # トークン化されたテキストをエンコードする
        encoded_texts = self.encoder(**tokenized_texts)
        
        # 最終層の[CLS]トークンのベクトルを取り出す ->  文ベクトルのスコア
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # mlp_only_trainがFlseや推論の場合はヘッドにベクトルを渡さずに返す。
        if self.mlp_only_train and not self.training:
            return encoded_texts


        # MLP層による変換
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.actiovation(encoded_texts)

        return encoded_texts


    def forward(
        self, 
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels : Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        
        # 各トークン化されたテキストを文ベクトルに変換
        encoded_texts_1 = self.encode_text(tokenized_texts_1)
        encoded_texts_2 = self.encode_text(tokenized_texts_2)

        # コサイン類似度を算出
        #  tokenized_texts_2.unsqueeze(0)はブロードキャストを利用？？
        # output-> (batch_size, batch_sizeの大きさのscores): 類似度行列
        sim_matrix = F.cosine_similarity(encoded_texts_1.unsqueeze(1),encoded_texts_2.unsqueeze(0),dim=2,)
        
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # positive score
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(sim_matrix, positive_mask)

        
        return ModelOutput(loss=loss, scores=positive_scores)

In [99]:
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/config.json
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-v3",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c

In [100]:
# モデルの訓練と評価をするTrainerを準備
# 今回評価指標として、スペアマンの順位相関係数を使用

# 評価結果の計算関数
def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """モデルの予想スコアと評価用スコアのスピアマン順位相関係数を計算"""

    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).statistic

    return {'spearman': spearman}

In [101]:
unsup_training_args = TrainingArguments(
    output_dir="outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=64,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=64,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to='all'
)

PyTorch: setting up devices


In [102]:
# 訓練セットと検証セットで異なるcollate関数を使用するのでカスタムtrainerを作成

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(self, eval_dataset: Dataset | None = None) -> DataLoader:

        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn,
            pin_memory=True
        )




In [103]:
# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(model=unsup_model,
                              args=unsup_training_args,
                              data_collator=unsup_train_collate_fn,
                              train_dataset=unsup_train_dataset,
                              eval_dataset=valid_dataset,
                              compute_metrics=compute_metrics
                             )

Using auto half precision backend


In [104]:
# 訓練の実行
unsup_trainer.train()

***** Running training *****
  Num examples = 1,000,000
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 15,625
  Number of trainable parameters = 111,797,760


Step,Training Loss,Validation Loss,Spearman
250,0.001900,2.516721,0.716522
500,0.000300,2.461056,0.735719
750,0.000400,2.478571,0.733308
1000,0.000200,2.458963,0.739426
1250,0.000300,2.500067,0.729833
1500,0.000200,2.474350,0.726624
1750,0.000100,2.490425,0.726557
2000,0.000000,2.448895,0.733077
2250,0.000300,2.381986,0.734255
2500,0.000300,2.427408,0.728923


***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-250
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_unsup_simcse/checkpoint-250] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-750
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_unsup_simcse/checkpoint-500] due to args.save_total

TrainOutput(global_step=15625, training_loss=0.00012896459624171257, metrics={'train_runtime': 2971.4218, 'train_samples_per_second': 336.539, 'train_steps_per_second': 5.258, 'total_flos': 0.0, 'train_loss': 0.00012896459624171257, 'epoch': 1.0})

In [105]:
# モデルの評価
unsup_trainer.evaluate(valid_dataset)

***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64


{'eval_loss': 2.2688992023468018,
 'eval_spearman': 0.7576461344472822,
 'eval_runtime': 9.7325,
 'eval_samples_per_second': 1279.319,
 'eval_steps_per_second': 20.036,
 'epoch': 1.0}

In [106]:
unsup_trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 1457
  Batch size = 64


{'eval_loss': 2.139267683029175,
 'eval_spearman': 0.7969628926980149,
 'eval_runtime': 1.3159,
 'eval_samples_per_second': 1107.189,
 'eval_steps_per_second': 17.478,
 'epoch': 1.0}

In [107]:
encoder_path = "outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

Configuration saved in outputs_unsup_simcse/encoder/config.json
Model weights saved in outputs_unsup_simcse/encoder/model.safetensors
tokenizer config file saved in outputs_unsup_simcse/encoder/tokenizer_config.json
Special tokens file saved in outputs_unsup_simcse/encoder/special_tokens_map.json


('outputs_unsup_simcse/encoder/tokenizer_config.json',
 'outputs_unsup_simcse/encoder/special_tokens_map.json',
 'outputs_unsup_simcse/encoder/vocab.txt',
 'outputs_unsup_simcse/encoder/added_tokens.json')